In [1]:
import requests
import os
import urllib.parse
import posixpath
import pandas as pd
import paramiko

In [10]:
# make course an object
class course():
    
    def __init__(self, canvasHostName, courseID):
        self.canvasHostName = canvasHostName
        self.courseID = courseID
        print('Create class with Canvas host name:{0} and course ID: {1})'.
              format(self.canvasHostName, self.courseID))
        
    def get_student_ids(self):
        '''Read Canvas authentication token from an environment variable, takes a
        Canvas host name (includes https://) and the Canvas course id and returns
        a list of the student id's of all students currently enrolled in the course.
    
        Example:
        course.get_student_ids()'''
        canvas_token = os.environ["CANVAS_TOKEN"]
        url_path = posixpath.join("api", "v1", "courses", self.courseID, "enrollments")
        api_url = urllib.parse.urljoin(self.canvasHostName, url_path)
        resp = requests.get(
              url = api_url,
              headers = {
                "Authorization": f"Bearer {canvas_token}",
                "Accept": "application/json+canvas-string-ids"
              },
              json={
                "enrollment_type": ["student"],
                "per_page": "500"
              },
            )
        students = resp.json()
        student_id = []
        for student in students:
            student_id.append(student['user_id'])
        return student_id
    
    def get_assignments(self):
        '''Read Canvas authentication token from an environment variable, takes a
        Canvas host name (includes https://) and the Canvas course id and returns
        a Pandas data frame with all existing assignments and their attributes/data

        Example:
        course.get_assignments()'''
        canvas_token = os.environ["CANVAS_TOKEN"]
        url_path = posixpath.join("api", "v1", "courses", self.courseID, "assignments")
        api_url = urllib.parse.urljoin(self.canvasHostName, url_path)
        resp = requests.get(
          url=api_url,
          headers={
            "Authorization": f"Bearer {canvas_token}",
            "Accept": "application/json+canvas-string-ids"
          },
          json={
            "per_page": "2000"
          },
        )
        assignments = resp.json()
        assign_data = pd.DataFrame.from_dict(assignments)
        return assign_data
    
    def get_assignment_due_date(self, assignment):
        '''Takes the name of a Canvas assignment and returns the due date.
        
        Example:
        course.get_assignment_due_date('worksheet_01')'''
        assignments = self.get_assignments()
        assignment = assignments[['name', 'due_at']].query('name == @assignment')
        due_date = assignment['due_at'].to_numpy()[0]
        due_date = due_date.replace("T", "-")
        due_date = due_date.replace(":", "-")
        return due_date[:16]

In [8]:
# All the settings for the assignment
assignment = 'worksheet_01'
due_day = 'sat' # should be sat or wed
grader = 'timberst' # should be graders cwl

# All the settings for the course 
dsci100 = course("https://canvas.ubc.ca", "40616")
students = dsci100.get_student_ids()
due_date = dsci100.get_assignment_due_date(assignment)[:14]
course_storage_path = '/tank/home/dsci100'
stu_repo_name = 'dsci-100'
ins_repo_name = 'dsci-100-instructor'
assignment_release_path = 'materials'
snapshot_prefix = 'zfs-auto-snap_' + due_day + '-'
snapshot_delay = '10'

# Create copy from and to paths
copy_from_path = os.path.join('.zfs', 'snapshot', snapshot_prefix + due_date + snapshot_delay)
copy_to_path = os.path.join(course_storage_path, grader, ins_repo_name, 'submitted')

Create class with Canvas host name:https://canvas.ubc.ca and course ID: 40616)


In [9]:
student_path_remote = os.path.join(course_storage_path, str(students[0]))
student_path

'/tank/home/dsci100/313452'

In [17]:
assignment_path = os.path.join(student_path_remote, copy_from_path, stu_repo_name, assignment_release_path, assignment, assignment + '.ipynb')
assignment_path

'/tank/home/dsci100/313452/.zfs/snapshot/zfs-auto-snap_sat-2019-09-08-01-10/dsci-100/materials/worksheet_01/worksheet_01.ipynb'

In [25]:
student_path_local = os.path.join(copy_to_path, str(students[0]))
student_path_local

'/tank/home/dsci100/timberst/dsci-100-instructor/submitted/313452'

In [24]:
submission_path = os.path.join(student_path_local, assignment, assignment + '.ipynb')
submission_path

'/tank/home/dsci100/timberst/dsci-100-instructor/submitted/submitted/313452/worksheet_01/worksheet_01.ipynb'

In [19]:
ssh = paramiko.SSHClient() 
ssh.load_host_keys(os.path.expanduser(os.path.join("~", ".ssh", "known_hosts")))
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect("hub-prod-dsci.stat.ubc.ca", username="stty2u")
sftp = ssh.open_sftp()

//anaconda3/lib/python3.7/site-packages/paramiko/ecdsakey.py:164: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.ecdsa_curve.curve_class(), pointinfo
//anaconda3/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
//anaconda3/lib/python3.7/site-packages/paramiko/kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.curve, Q_S_bytes
//anaconda3/lib/python3.7/site-packages/paramik

In [21]:
if not os.path.exists(student_path_local):
    os.mkdir(student_path_local)
    os.mkdir(student_path_local, assignment)  
else:   
    if not os.path.exists(os.path.join(student_path_local, assignment)):
        os.mkdir(student_path_local, assignment)

In [ ]:
sftp.get(remotepath=assignment_path, localpath=submission_path)
sftp.close()
ssh.close()

In [ ]:
# looping over student id
for student in students:
    student_path = os.path.join(course_storage_path, str(student))

    assignment_path = os.path.join(student_path, copy_from_path, stu_repo_name, assignment_release_path, assignment)

    submission_path = os.path.join(copy_to_path, 'submitted', str(student), assignment)
    try:
      shutil.copytree(assignment_path, submission_path)
    except:
      pass